In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import vectorbt as vbt
import yfinance as yf
import plotly.graph_objects as go

In [22]:
btc = pd.read_csv('btcusd_30min_data.csv', index_col=0, parse_dates=True)
btc

,Open,High,Low,Close
Timestamp,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58
2012-01-01 10:30:00,4.58,4.58,4.58,4.58
2012-01-01 11:00:00,4.58,4.58,4.58,4.58
2012-01-01 11:30:00,4.58,4.58,4.58,4.58
2012-01-01 12:00:00,4.58,4.58,4.58,4.58
...,...,...,...,...
2025-05-14 22:30:00,103555.00,103556.00,103457.00,103535.00
2025-05-14 23:00:00,103507.00,103579.00,103422.00,103579.00
2025-05-14 23:30:00,103580.00,103621.00,103526.00,103531.00


In [24]:
btc = btc[btc.index >= '2016-01-01']
btc.index = pd.to_datetime(btc.index).strftime('%Y-%m-%d %H:%M')
df = pd.DataFrame(btc)
df

,Open,High,Low,Close
Timestamp,,,,
2016-01-01 00:00,430.91,430.99,429.86,430.97
2016-01-01 00:30,430.97,430.97,429.77,430.01
2016-01-01 01:00,430.01,431.90,430.00,431.89
2016-01-01 01:30,431.89,431.90,430.75,431.88
2016-01-01 02:00,431.88,433.89,431.67,432.01
...,...,...,...,...
2025-05-14 22:30,103555.00,103556.00,103457.00,103535.00
2025-05-14 23:00,103507.00,103579.00,103422.00,103579.00
2025-05-14 23:30,103580.00,103621.00,103526.00,103531.00


In [26]:
period = 100

df['SwingHigh'] = df['High'].shift(1).rolling(window=period, min_periods=1).max()
df['SwingLow'] = df['Low'].shift(1).rolling(window=period, min_periods=1).min()


# Conditions for swing failure signals
df['BearishSignal'] = (df['High'] > df['SwingHigh']) & (df['Close'] < df['SwingHigh'])
df['BullishSignal'] = (df['Low'] < df['SwingLow']) & (df['Close'] > df['SwingLow'])
df.head(100)

,Open,High,Low,Close,SwingHigh,SwingLow,BearishSignal,BullishSignal
Timestamp,,,,,,,,
2016-01-01 00:00,430.91,430.99,429.86,430.97,NaN,NaN,False,False
2016-01-01 00:30,430.97,430.97,429.77,430.01,430.99,429.86,False,True
2016-01-01 01:00,430.01,431.90,430.00,431.89,430.99,429.77,False,False
2016-01-01 01:30,431.89,431.90,430.75,431.88,431.90,429.77,False,False
2016-01-01 02:00,431.88,433.89,431.67,432.01,431.90,429.77,False,False
...,...,...,...,...,...,...,...,...
2016-01-02 23:30,433.16,434.25,433.08,433.47,436.00,427.20,False,False
2016-01-03 00:00,433.68,433.79,432.05,433.42,436.00,427.20,False,False
2016-01-03 00:30,433.42,433.75,432.27,433.18,436.00,427.20,False,False


In [4]:
btc['High_Swing'] = btc['High'][(btc['High'].shift(1) < btc['High']) & (btc['High'].shift(-1) < btc['High'])]
btc['Low_Swing'] = btc['Low'][(btc['Low'].shift(1) > btc['Low']) & (btc['Low'].shift(-1) > btc['Low'])]
btc


,Open,High,Low,Close,High_Swing,Low_Swing
Timestamp,,,,,,
2016-01-01 00:00:00,430.91,430.99,429.86,430.97,NaN,NaN
2016-01-01 00:30:00,430.97,430.97,429.77,430.01,NaN,429.77
2016-01-01 01:00:00,430.01,431.90,430.00,431.89,NaN,NaN
2016-01-01 01:30:00,431.89,431.90,430.75,431.88,NaN,NaN
2016-01-01 02:00:00,431.88,433.89,431.67,432.01,433.89,NaN
...,...,...,...,...,...,...
2025-05-14 22:30:00,103555.00,103556.00,103457.00,103535.00,NaN,NaN
2025-05-14 23:00:00,103507.00,103579.00,103422.00,103579.00,NaN,103422.00
2025-05-14 23:30:00,103580.00,103621.00,103526.00,103531.00,NaN,NaN


In [16]:
df["High_Swing"] = (df["High"] > df["High"].shift(1)) & (df["High"] > df["High"].shift(-1))
df["Low_Swing"] = (df["Low"] < df["Low"].shift(1)) & (df["Low"] < df["Low"].shift(-1))
print(df.tail(10))


                      Open      High       Low     Close  High_Swing  \
Timestamp                                                              
2025-05-14 20:00  103325.0  103507.0  103314.0  103451.0       False   
2025-05-14 20:30  103469.0  103694.0  103402.0  103576.0        True   
2025-05-14 21:00  103579.0  103608.0  103459.0  103563.0       False   
2025-05-14 21:30  103563.0  103670.0  103465.0  103662.0       False   
2025-05-14 22:00  103662.0  103670.0  103500.0  103552.0       False   
2025-05-14 22:30  103555.0  103556.0  103457.0  103535.0       False   
2025-05-14 23:00  103507.0  103579.0  103422.0  103579.0       False   
2025-05-14 23:30  103580.0  103621.0  103526.0  103531.0       False   
2025-05-15 00:00  103514.0  103845.0  103514.0  103794.0       False   
2025-05-15 00:30  103789.0  103870.0  103724.0  103869.0       False   

                  Low_Swing  
Timestamp                    
2025-05-14 20:00      False  
2025-05-14 20:30      False  
2025-05-14 21:0

In [6]:
# Detect Swing Failure Highs
df["Swing_Failure_High"] = (
    (df["High"] > df["High_Swing"].shift(1))  
    & (df["Open"] < df["High_Swing"].shift(1))  
    & (df["Close"] < df["High_Swing"].shift(1)) 
)
# Detect Swing Failure Lows
df["Swing_Failure_Low"] = (
    (df["Low"] < df["Low_Swing"].shift(1))  
    & (df["Open"] > df["Low_Swing"].shift(1)) 
    & (df["Close"] > df["Low_Swing"].shift(1)) 
)

print(df[[ "Swing_Failure_High", "Swing_Failure_Low"]])

                     Swing_Failure_High  Swing_Failure_Low
Timestamp                                                 
2012-01-01 10:00:00               False              False
2012-01-01 12:00:00               False              False
2012-01-01 14:00:00               False              False
2012-01-01 16:00:00               False              False
2012-01-01 18:00:00               False              False
...                                 ...                ...
2025-05-14 16:00:00               False              False
2025-05-14 18:00:00               False              False
2025-05-14 20:00:00               False              False
2025-05-14 22:00:00               False              False
2025-05-15 00:00:00               False              False

[58592 rows x 2 columns]


c:\MyDisk\python\myenv\Lib\site-packages\pandas\core\computation\expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
c:\MyDisk\python\myenv\Lib\site-packages\pandas\core\computation\expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
c:\MyDisk\python\myenv\Lib\site-packages\pandas\core\computation\expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
c:\MyDisk\python\myenv\Lib\site-packages\pandas\core\computation\expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


In [29]:
df["Lookback_Candles"] = None
for i in range(len(df)):
    try:
        current_high = df.iloc[i]["High"]
    except (KeyError, IndexError):
        continue
    lookback_count = 0
    # Search backwards for a candle where Open > current High
    for j in range(i - 1, -1, -1):  # Iterate backwards
        lookback_count += 1
        try:
            if df.iloc[j]["Open"] > current_high:
                df.iloc[i, df.columns.get_loc("Lookback_Candles")] = lookback_count
                break  # Stop once we find the lookback candle
        except (KeyError, IndexError):
            continue
# Display Results
print(df[["High", "Lookback_Candles"]])


                         High Lookback_Candles
Timestamp                                     
2025-05-06 18:00:00   95017.0             None
2025-05-06 20:00:00   94678.0                1
2025-05-06 22:00:00   96835.0             None
2025-05-07 00:00:00   97253.0             None
2025-05-07 02:00:00   96502.0                1
...                       ...              ...
2025-05-14 16:00:00  103282.0                1
2025-05-14 18:00:00  103325.0                2
2025-05-14 20:00:00  103669.0                4
2025-05-14 22:00:00  103531.0                1
2025-05-15 00:00:00  103870.0                6

[100 rows x 2 columns]
